In [1]:
import sys
sys.path.append('./venv/lib/python3.6/site-packages/')

import yaml
import weibopy as wb


with open('./credentials.yml') as f:
    cred = yaml.load(f)

oauth = wb.OAuthHandler(**cred['weibo']).authorize()
api = wb.API(oauth=oauth)
# data, status = api.statuses.user_timeline()

Please load the link https://api.weibo.com/oauth2/authorize?response_type=code&client_id=3677147061&redirect_uri=https%3A%2F%2Fwww.greenpeace.org%2F&state=02oxq90VsCcjJ15z9k1TB7HvHn1Ick to your browser and authorize weibo API access.
Enter the full callback URL: https://www.greenpeace.org/usa/?state=02oxq90VsCcjJ15z9k1TB7HvHn1Ick&code=53ead9f0a340ab591555cd11d728a930
UID: 1292378751
Access Token expires at: 2023/07/09 22:13:07


In [2]:
from utils import MongoWriter
from weibo_data import *
writer = MongoWriter(**cred['mongo'])

In [3]:
writer.get_since_id(collection='statuses')

0

In [4]:
def crawl_statuses(since_id=0, insert: bool=False):
    records = list()
    skip_id = set()
    max_id = 0
    while True:
        data, _ = api.statuses.user_timeline(since_id=since_id, max_id=max_id)
        records = [s for s in [Status(x) for x in data['statuses']] if s.get('_id') not in skip_id]
        print(len(records))
        if records:
            for record in records:
                if record.get('_id') not in skip_id:
                    print(record.get('_id'), record.get('created_at'))
                    if insert:
                        record.write(writer=writer)
                    skip_id.add(record.get('_id'))
            else:
                max_id = record.get('_id')
        else:
            break

In [9]:
def crawl_comments(sid, since_id=0, insert: bool=False):
    records = list()
    skip_id = set()
    max_id = 0
    while True:
        data, _ = api.comments.show(id=sid, since_id=since_id, max_id=max_id)
        records = [s for s in [Comment(x) for x in data['comments']] if s.get('_id') not in skip_id]
        print(len(records))
        if records:
            for record in records:
                if record.get('_id') not in skip_id:
                    print(record.get('_id'), record.get('created_at'))
                    if insert:
                        record.write(writer=writer)
                    skip_id.add(record.get('_id'))
            else:
                max_id = record.get('_id')
        else:
            break

In [10]:
crawl_comments(sid=4246117354311557, since_id=0, insert=True)

15
4247621042839473 Tue Jun 05 18:40:36 +0800 2018
4247591015228869 Tue Jun 05 16:41:17 +0800 2018
4247589609939666 Tue Jun 05 16:35:42 +0800 2018


ERROR:root:E11000 duplicate key error collection: weibo_data.comments index: _id_ dup key: { : 4247589609939666 }


4247588854679883 Tue Jun 05 16:32:42 +0800 2018
4247587849107579 Tue Jun 05 16:28:42 +0800 2018


ERROR:root:E11000 duplicate key error collection: weibo_data.users index: _id_ dup key: { : 6508355557 }


4247586757668297 Tue Jun 05 16:24:22 +0800 2018
4247585336284492 Tue Jun 05 16:18:43 +0800 2018
4247585038101360 Tue Jun 05 16:17:31 +0800 2018
4247584165799887 Tue Jun 05 16:14:04 +0800 2018
4247569318554993 Tue Jun 05 15:15:04 +0800 2018
4247561815154053 Tue Jun 05 14:45:14 +0800 2018
4247558542610850 Tue Jun 05 14:32:14 +0800 2018
4247556907409410 Tue Jun 05 14:25:45 +0800 2018


ERROR:root:E11000 duplicate key error collection: weibo_data.users index: _id_ dup key: { : 1256234111 }


4246713642073295 Sun Jun 03 06:34:55 +0800 2018
4246563519743440 Sat Jun 02 20:38:22 +0800 2018
0


In [ ]:
data, _ = api.statuses.user_timeline()

In [ ]:
skip_id = set()
records = [s for s in [Status(x) for x in data['statuses']] if s.get('_id') not in skip_id]

In [ ]:
records[0]._data['_id']

In [5]:
crawl_statuses(since_id=0, insert=True)

5
4250426327276250 Wed Jun 13 12:27:48 +0800 2018
4257970281382180 Wed Jul 04 08:04:48 +0800 2018


ERROR:root:E11000 duplicate key error collection: weibo_data.users index: _id_ dup key: { : 1292378751 }


4251242778027475 Fri Jun 15 18:32:05 +0800 2018


ERROR:root:E11000 duplicate key error collection: weibo_data.users index: _id_ dup key: { : 1292378751 }


4248940779458347 Sat Jun 09 10:04:47 +0800 2018


...

4243617021947926 Fri May 25 17:30:03 +0800 2018


ERROR:root:E11000 duplicate key error collection: weibo_data.users index: _id_ dup key: { : 1292378751 }


4241343654479917 Sat May 19 10:56:29 +0800 2018


ERROR:root:E11000 duplicate key error collection: weibo_data.users index: _id_ dup key: { : 1292378751 }


0


In [ ]:
crawl_statuses(since_id=4243617021947926)

In [ ]:
for s_data in data['statuses']:
    status = Status(s_data)
    
    print(status.get('_id'), status.get('created_at'))
    print(len(status._data.keys()), list(status._data.keys()))
#     status.write(writer=writer) 

In [ ]:
data2, status = api.statuses.user_timeline(max_id=4248766729509158)

In [ ]:
for s_data in data2['statuses']:
    status = Status(s_data)
    print(status.get('_id'), status.get('created_at'))
    print(len(status._data.keys()), list(status._data.keys()))

In [ ]:
s_data

In [ ]:
'created_at', 'can_edit', 'text', 'textLength', 'source_allowclick',
'source_type', 'source', 'favorited', 'truncated', 'pic_urls', 'thumbnail_pic', 'bmiddle_pic', 'original_pic', 'geo',
'is_paid', 'mblog_vip_type', 'user', 'annotations', 'reposts_count', 'comments_count', 
'attitudes_count', 'pending_approval_count', 'isLongText', 'multi_attitude', 'hide_flag', 'mlevel',
'visible', 'biz_ids', 'biz_feature', 'page_type', 'hasActionTypeCard', 'darwin_tags', 'hot_weibo_tags', 
'text_tag_tips', 'userType', 'extend_info', 'more_info_type', 'positive_recom_flag', 'content_auth', 'gif_ids',
'is_show_bulletin', 'comment_manage_info'

'id', 'idstr', 'mid', 'in_reply_to_status_id', 'in_reply_to_user_id',
'in_reply_to_screen_name',

In [ ]:
data3, status = api.statuses.user_timeline(max_id=0)

In [ ]:
data3

In [ ]:
for s_data in data3['statuses']:
    status = Status(s_data)
    print(status.get('_id'), status.get('created_at'))

In [ ]:
c_data, _ = api.comments.show(id=4246117354311557)

In [ ]:
for c in c_data['comments']:
    comment = Comment(c)
    print(comment.get('_id'), comment.get('created_at'), comment.get('reply_id'))
    comment.write(writer=writer, write_obj=True) 

In [ ]:
c_data2, _ = api.comments.show(id=4241343654479917, page=2)
for c in c_data2['comments']:
    comment = Comment(c)
    print(comment.get('_id'), comment.get('created_at'), comment.get('reply_id'))
    status.write(writer=writer) 

In [ ]:
# data, status = api.comments.show(id=4250426327276250)

for s_data in data['comments']:
    print(s_data['id'], s_data['floor_number'], s_data['created_at'], s_data['status']['id'], s_data.get('reply_comment', {}).get('id'))
#     s_data['text']

In [ ]:
data, _ = api.users.show(uid=2625796571)

In [ ]:
data

In [ ]:
from weibo_data import *
from utils import MongoWriter


def main(cred):
    writer = MongoWriter(**cred['mongo'])
    oauth = wb.OAuthHandler(**cred['weibo']).authorize()
    api = wb.API(oauth=oauth)
    data, _ = api.statuses.user_timeline()
    for s_data in data['statuses']:
        status = Status(s_data)
        print(status)
        status.write(writer=writer)


if __name__ == '__main__':
    with open('./credentials.yml') as f:
        cred = yaml.load(f)
    main(cred=cred)

In [ ]:
api = wb.API(oauth)
data, status = api.statuses.user_timeline(count=100)

In [ ]:
statuses = data.get('statuses', [])

In [ ]:
for status in statuses:
    print(status.get('created_at'), status.get('id'))

In [ ]:
data, status = api.statuses.user_timeline(count=100, max_id=4248766729509158, trim_user=1)

In [ ]:
statuses2 = data.get('statuses', [])

In [ ]:
for status in statuses2:
    print(status.get('created_at'), status.get('id'))

In [ ]:
status = statuses2[2]
status

In [ ]:
status.keys()

In [ ]:
data, _ = api.statuses.show(id='4243617021947926')

In [ ]:
data.keys()

In [ ]:
data['user']

In [ ]:
data, _ = api.users.show(uid='1292378751')

In [ ]:
data

In [ ]:
help(api.users.show)

In [ ]:
'screen_name', 'name', 'province', 'city', 'location', 'description', 'url', 'profile_image_url', 
'cover_image', 'cover_image_phone', 'profile_url', 'domain', 'gender', 'followers_count', 'friends_count', 
'pagefriends_count', 'statuses_count', 'video_status_count', 'favourites_count', 'created_at', 'geo_enabled',
'verified', 'insecurity', 'ptype', 'allow_all_comment', 'avatar_large', 'avatar_hd', 'verified_reason', 
'verified_state', 'verified_level', 'verified_type_ext', 'has_service_tel', 'verified_contact_name', 
'verified_contact_email', 'verified_contact_mobile', 'follow_me', 'like', 'like_me', 'online_status', 'bi_followers_count', 'lang', 'star', 'mbtype', 'mbrank', 'block_word', 'block_app', 'credit_score', 'user_ability', 'urank', 'story_read_state', 'vclub_member'

In [ ]:
'screen_name', 'name', 'province', 'city', 'location', 'description', 'url', 'profile_image_url',
'cover_image', 'cover_image_phone', 'profile_url', 'domain', 'gender', 'followers_count', 'friends_count',
'pagefriends_count', 'statuses_count', 'video_status_count', 'favourites_count', 'created_at', 'geo_enabled', 
'verified', 'insecurity', 'ptype', 'allow_all_comment', 'avatar_large', 'avatar_hd', 'verified_reason', 
'verified_state', 'verified_level', 'verified_type_ext', 'has_service_tel','verified_contact_name',
'verified_contact_email', 'verified_contact_mobile', 'follow_me', 'like', \
            'like_me', 'online_status', 'bi_followers_count', 'lang', 'star', 'block_word', 'block_app', 'credit_score',\
            'user_ability', 'urank', 'story_read_state',

In [11]:
c_1, _ = api.comments.show(id=4246117354311557)

In [12]:
for c in c_1['comments']:
    print(c['created_at'], c['id'])

Tue Jun 05 18:40:36 +0800 2018 4247621042839473
Tue Jun 05 16:41:17 +0800 2018 4247591015228869
Tue Jun 05 16:35:42 +0800 2018 4247589609939666
Tue Jun 05 16:32:42 +0800 2018 4247588854679883
Tue Jun 05 16:28:42 +0800 2018 4247587849107579
Tue Jun 05 16:24:22 +0800 2018 4247586757668297
Tue Jun 05 16:18:43 +0800 2018 4247585336284492
Tue Jun 05 16:17:31 +0800 2018 4247585038101360
Tue Jun 05 16:14:04 +0800 2018 4247584165799887
Tue Jun 05 15:15:04 +0800 2018 4247569318554993
Tue Jun 05 14:45:14 +0800 2018 4247561815154053
Tue Jun 05 14:32:14 +0800 2018 4247558542610850
Tue Jun 05 14:25:45 +0800 2018 4247556907409410
Sun Jun 03 06:34:55 +0800 2018 4246713642073295
Sat Jun 02 20:38:22 +0800 2018 4246563519743440


In [13]:
c_2, _ = api.comments.show(id=4246117354311557, max_id=4246563519743440)

In [14]:
for c in c_2['comments']:
    print(c['created_at'], c['id'])

Sat Jun 02 20:38:22 +0800 2018 4246563519743440


In [ ]:
c_3, _ = api.comments.show(id=4246117354311557, max_id=4246563519743440)

In [ ]:
for c in comments:
    print(c['created_at'], c['id'])

In [ ]:
data2, _ = api.comments.show(id='4250426327276250', max_id='4250427878887380')

In [ ]:
comments = data2['comments']

for c in comments:
    print(c['created_at'], c['id'])